In [ ]:
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

user_data = pd.read_csv("/Users/kayleecho/Documents/data science projects/recommendation_system/synthetic_user_data.csv", sep=",")
print(user_data.shape)
user_data.head(10)

(1000, 23)


,user_id,investment_goal_long_term_growth,investment_goal_stable_income,investment_goal_balanced_approach,investment_goal_high_risk_high_reward,investment_goal_short_term_trading,risk_tolerance,holding_period_response,holding_period_reaction_strength,expected_return_score,product_preference_stocks,product_preference_etfs,product_preference_government_bonds,product_preference_corporate_bonds,product_preference_municipal_bonds,product_preference_savings_bonds,product_preference_derivatives,product_preference_alternative_investments,factor_weight_investment_goals,factor_weight_risk_tolerance,factor_weight_holding_period,factor_weight_return_score,factor_weight_product_preference
0,1,0.29,0.24,0.29,0.14,0.04,4.83,3.33,7.39,3.15,18.72,17.62,5.92,11.99,18.77,15.60,11.07,0.31,22.43,20.48,29.33,2.26,25.50
1,2,0.04,0.38,0.06,0.33,0.19,5.04,6.67,8.58,4.23,16.37,15.63,8.91,15.34,8.18,15.55,16.97,3.05,6.26,7.80,22.03,22.12,41.79
2,3,0.39,0.02,0.15,0.17,0.27,7.28,3.33,1.53,2.09,12.67,20.59,8.26,19.86,11.26,10.51,1.75,15.10,40.52,9.34,29.33,2.30,18.51
3,4,0.32,0.12,0.30,0.03,0.23,2.97,10.00,6.59,7.55,11.44,8.32,2.64,18.60,20.01,14.52,13.44,11.03,7.04,7.09,31.62,23.45,30.80
4,5,0.18,0.18,0.01,0.27,0.36,5.61,1.00,5.57,9.24,11.17,8.25,20.71,19.18,1.54,23.51,7.33,8.31,17.70,14.20,12.73,25.42,29.95
5,6,0.11,0.43,0.13,0.02,0.31,2.27,1.00,1.54,2.43,12.85,15.70,8.81,9.67,9.54,14.29,14.49,14.65,8.65,10.75,19.35,39.45,21.80
6,7,0.09,0.14,0.26,0.28,0.23,1.32,6.67,6.57,4.58,6.13,15.85,18.69,18.37,18.39,8.98,10.59,3.00,41.30,3.19,16.33,0.81,38.37
7,8,0.34,0.15,0.21,0.29,0.01,8.80,10.00,1.02,3.78,9.20,10.97,14.08,18.54,20.23,11.55,7.98,7.45,13.69,37.73,9.77,33.06,5.75
8,9,0.01,0.11,0.22,0.41,0.25,5.55,3.33,3.81,3.95,3.92,9.29,11.56,14.99,16.33,17.70,17.63,8.58,9.11,35.83,0.85,44.64,9.57
9,10,0.18,0.37,0.26,0.16,0.03,3.06,6.67,5.90,9.87,26.20,9.83,4.60,21.51,1.00,8.58,4.51,23.77,25.92,33.34,4.90,20.51,15.33


In [2]:
new_users = user_data[0:19]
old_users = user_data[20:]

user_data.columns

Index(['user_id', 'investment_goal_long_term_growth',
       'investment_goal_stable_income', 'investment_goal_balanced_approach',
       'investment_goal_high_risk_high_reward',
       'investment_goal_short_term_trading', 'risk_tolerance',
       'holding_period_response', 'holding_period_reaction_strength',
       'expected_return_score', 'product_preference_stocks',
       'product_preference_etfs', 'product_preference_government_bonds',
       'product_preference_corporate_bonds',
       'product_preference_municipal_bonds',
       'product_preference_savings_bonds', 'product_preference_derivatives',
       'product_preference_alternative_investments',
       'factor_weight_investment_goals', 'factor_weight_risk_tolerance',
       'factor_weight_holding_period', 'factor_weight_return_score',
       'factor_weight_product_preference'],
      dtype='object')

## Let's Try our second method first which is using multidimensional similarity measures.

In [3]:
new_users = new_users.drop(['user_id', 'factor_weight_investment_goals', 'factor_weight_risk_tolerance', 'factor_weight_holding_period', 'factor_weight_return_score', 'factor_weight_product_preference'], axis=1)
old_users = old_users.drop(['user_id', 'factor_weight_investment_goals', 'factor_weight_risk_tolerance', 'factor_weight_holding_period', 'factor_weight_return_score', 'factor_weight_product_preference'], axis=1)
print(new_users.columns)
print(old_users.columns)
print(new_users.shape)
print(old_users.shape)

Index(['investment_goal_long_term_growth', 'investment_goal_stable_income',
       'investment_goal_balanced_approach',
       'investment_goal_high_risk_high_reward',
       'investment_goal_short_term_trading', 'risk_tolerance',
       'holding_period_response', 'holding_period_reaction_strength',
       'expected_return_score', 'product_preference_stocks',
       'product_preference_etfs', 'product_preference_government_bonds',
       'product_preference_corporate_bonds',
       'product_preference_municipal_bonds',
       'product_preference_savings_bonds', 'product_preference_derivatives',
       'product_preference_alternative_investments'],
      dtype='object')
Index(['investment_goal_long_term_growth', 'investment_goal_stable_income',
       'investment_goal_balanced_approach',
       'investment_goal_high_risk_high_reward',
       'investment_goal_short_term_trading', 'risk_tolerance',
       'holding_period_response', 'holding_period_reaction_strength',
       'expected_retu

## This is normally how you would calculate cosine similarity using numpy!

In [4]:
import numpy as np
from numpy.linalg import norm

A = np.array([0.40, 0.10, 0.40, 0.05, 0.05, 0.75, 0.667, 0.700, 0.85, 0.40, 0.20, 0.15, 0.05, 0.05, 0.10, 0.05])
B = np.array([0.70, 0.10, 0.10, 0.05, 0.05, 0.625, 1.00, 0.85, 0.725, 0.50, 0.10, 0.10, 0.05, 0.05, 0.10, 0.10])
C = np.array([0.10, 0.10, 0.20, 0.50, 0.10, 0.875, 0.333, 0.50, 0.90, 0.30, 0.15, 0.15, 0.10, 0.10, 0.15, 0.15])

print("A:", A)
print("B:", B)
print("C:", C)
 
# compute cosine similarity
cosine_ab = np.dot(A,B)/(norm(A)*norm(B))
cosine_ac = np.dot(A,C)/(norm(A)*norm(C))
print("Cosine Similarity:", cosine_ab)
print("Cosine Similarity:", cosine_ac)

A: [0.4   0.1   0.4   0.05  0.05  0.75  0.667 0.7   0.85  0.4   0.2   0.15
 0.05  0.05  0.1   0.05 ]
B: [0.7   0.1   0.1   0.05  0.05  0.625 1.    0.85  0.725 0.5   0.1   0.1
 0.05  0.05  0.1   0.1  ]
C: [0.1   0.1   0.2   0.5   0.1   0.875 0.333 0.5   0.9   0.3   0.15  0.15
 0.1   0.1   0.15  0.15 ]
Cosine Similarity: 0.9459992837365002
Cosine Similarity: 0.9000536145871202


## Specifically for our case: calculate cosine similarity

In [5]:
import numpy as np
from numpy.linalg import norm
import pandas as pd

new_users = new_users.to_numpy()
old_users = old_users.to_numpy()

cosine_similarities = {}

for i in range(len(new_users)):
    similarities = []
    for j in range(len(old_users)):
        cosine_similarity = np.dot(new_users[i], old_users[j]) / (norm(new_users[i]) * norm(old_users[j]))
        similarities.append(cosine_similarity)
    cosine_similarities[f"User_{i+1}"] = similarities

cosine_sim_df = pd.DataFrame.from_dict(cosine_similarities, orient='index')

print(cosine_sim_df.shape)
cosine_sim_df

(19, 980)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979
User_1,0.646780,0.705349,0.885670,0.750899,0.542893,0.928692,0.797425,0.665439,0.821752,0.794411,0.686605,0.755189,0.851475,0.664331,0.886615,0.747447,0.739509,0.938726,0.789263,0.825181,0.849470,0.797073,0.830365,0.773858,0.922624,0.857999,0.720211,0.810227,0.769730,0.650036,0.925109,0.853630,0.889969,0.563192,0.807326,0.811327,0.696093,0.786274,0.784329,0.596671,0.759133,0.863090,0.713860,0.897840,0.808675,0.720558,0.863707,0.767124,0.775432,0.936059,0.918858,0.958533,0.877434,0.900705,0.850043,0.708091,0.678492,0.729326,0.861496,0.901607,0.620028,0.875212,0.880267,0.803373,0.884261,0.803806,0.768533,0.777646,0.675029,0.745111,0.784949,0.901940,0.743284,0.657001,0.853116,0.769982,0.882020,0.876902,0.903312,0.668639,0.643155,0.827238,0.833451,0.737370,0.868585,0.666221,0.983930,0.663923,0.738762,0.641545,0.742708,0.808589,0.868093,0.781251,0.661333,0.782272,0.826874,0.630567,0.758531,0.784371,0.741596,0.795943,0.693006,0.928157,0.758248,0.775476,0.799926,0.655108,0.809423,0.878219,0.758354,0.768185,0.880191,0.879422,0.838984,0.722573,0.845975,0.891464,0.826443,0.702950,0.754316,0.634929,0.846295,0.749101,0.892761,0.690840,0.692978,0.929483,0.765323,0.918332,0.742164,0.914417,0.950367,0.760241,0.749605,0.865746,0.889398,0.810563,0.703005,0.904811,0.783560,0.816045,0.623561,0.825763,0.902384,0.819450,0.703911,0.627812,0.669858,0.761573,0.803122,0.779411,0.719167,0.785348,0.837307,0.693690,0.888501,0.861410,0.697614,0.875840,0.876364,0.906292,0.694759,0.763885,0.926453,0.797509,0.855619,0.833456,0.809585,0.835668,0.729874,0.648171,0.799344,0.798757,0.812530,0.697252,0.821026,0.862778,0.790599,0.797982,0.697523,0.812789,0.866479,0.860789,0.792251,0.927056,0.697431,0.726245,0.911533,0.724037,0.592924,0.749271,0.778725,0.800139,0.807494,0.667096,0.914308,0.697443,0.825899,0.671697,0.729608,0.820067,0.685377,0.672520,0.845140,0.846487,0.684427,0.543712,0.831473,0.949340,0.831003,0.763285,0.759720,0.533508,0.802903,0.706713,0.908357,0.751317,0.741395,0.771014,0.862227,0.815010,0.876002,0.861242,0.832529,0.801187,0.856533,0.736106,0.838224,0.744595,0.778098,0.727633,0.799567,0

In [6]:
cosine_sim_df_transposed = cosine_sim_df.T
print(cosine_sim_df_transposed.shape)
cosine_sim_df_transposed.head(10)

(980, 19)


,User_1,User_2,User_3,User_4,User_5,User_6,User_7,User_8,User_9,User_10,User_11,User_12,User_13,User_14,User_15,User_16,User_17,User_18,User_19
0,0.646780,0.664223,0.774081,0.818190,0.720922,0.785634,0.663771,0.752609,0.789602,0.705911,0.818278,0.687851,0.555404,0.728254,0.651525,0.731310,0.694780,0.412389,0.560348
1,0.705349,0.671111,0.818284,0.674707,0.648949,0.824023,0.723641,0.766952,0.684952,0.716362,0.858438,0.525258,0.575670,0.677768,0.777368,0.822406,0.766487,0.761880,0.728751
2,0.885670,0.894914,0.803140,0.882047,0.723963,0.886742,0.852763,0.874620,0.880462,0.742157,0.867034,0.763681,0.689442,0.689013,0.919893,0.888417,0.877043,0.680343,0.696366
3,0.750899,0.783612,0.864367,0.760602,0.874438,0.863545,0.843971,0.846135,0.818131,0.733189,0.874588,0.641478,0.768573,0.893258,0.766802,0.786467,0.785472,0.678239,0.831159
4,0.542893,0.652424,0.786401,0.604008,0.833017,0.797613,0.688338,0.698280,0.739923,0.674235,0.747880,0.586974,0.724484,0.785845,0.701217,0.684224,0.598024,0.566282,0.795748
5,0.928692,0.907447,0.775371,0.824650,0.625708,0.852791,0.852205,0.826294,0.790223,0.667933,0.834530,0.669687,0.681647,0.667190,0.777400,0.890334,0.898279,0.783856,0.770933
6,0.797425,0.786290,0.758335,0.683368,0.734087,0.739012,0.710485,0.733952,0.591353,0.818196,0.697879,0.680818,0.641523,0.609356,0.774203,0.679388,0.840431,0.845837,0.658452
7,0.665439,0.785012,0.648232,0.577341,0.739374,0.800314,0.747662,0.670639,0.748049,0.514900,0.768379,0.559170,0.696645,0.741818,0.650444,0.790103,0.610379,0.667469,0.947637
8,0.821752,0.895936,0.804735,0.870128,0.800152,0.958357,0.732859,0.766704,0.866880,0.835188,0.892652,0.897005,0.635250,0.684567,0.798438,0.908225,0.807538,0.690708,0.782126
9,0.794411,0.865743,0.835167,0.746053,0.862352,0.913389,0.801265,0.771540,0.813015,0.759043,0.873032,0.718147,0.768600,0.813794,0.737886,0.872660,0.790320,0.797757,0.926494


In [7]:
cosine_sim_sorted = pd.DataFrame()

for i in cosine_sim_df_transposed.columns:
    sorted_indices = cosine_sim_df_transposed[i].sort_values(ascending=False).index
    cosine_sim_sorted[i] = cosine_sim_df_transposed[i].sort_values(ascending=False).values
    cosine_sim_sorted["old_user_id_for_" + i] = cosine_sim_df_transposed.index[sorted_indices].values

cosine_sim_sorted

,User_1,old_user_id_for_User_1,User_2,old_user_id_for_User_2,User_3,old_user_id_for_User_3,User_4,old_user_id_for_User_4,User_5,old_user_id_for_User_5,User_6,old_user_id_for_User_6,User_7,old_user_id_for_User_7,User_8,old_user_id_for_User_8,User_9,old_user_id_for_User_9,User_10,old_user_id_for_User_10,User_11,old_user_id_for_User_11,User_12,old_user_id_for_User_12,User_13,old_user_id_for_User_13,User_14,old_user_id_for_User_14,User_15,old_user_id_for_User_15,User_16,old_user_id_for_User_16,User_17,old_user_id_for_User_17,User_18,old_user_id_for_User_18,User_19,old_user_id_for_User_19
0,0.983930,86,0.974706,103,0.971487,595,0.975662,512,0.958621,943,0.974619,64,0.957429,830,0.974191,489,0.964483,525,0.944660,629,0.973707,837,0.970920,67,0.962462,809,0.961399,799,0.958533,151,0.964750,547,0.955106,51,0.957544,83,0.950060,281
1,0.974736,367,0.971896,923,0.965622,729,0.967715,445,0.958093,897,0.971752,822,0.954613,921,0.955929,921,0.964298,156,0.940887,657,0.971340,344,0.950874,802,0.957254,640,0.957515,747,0.949614,455,0.964057,225,0.954376,129,0.938269,395,0.948442,576
2,0.968051,960,0.967486,132,0.958193,771,0.967711,49,0.956617,561,0.963620,19,0.951930,489,0.953578,777,0.964093,118,0.933346,814,0.971124,325,0.945022,173,0.953177,408,0.956786,342,0.947021,292,0.960780,183,0.948123,960,0.932247,952,0.947637,7
3,0.958533,51,0.967349,527,0.953281,311,0.964549,571,0.949757,538,0.959237,469,0.950255,898,0.950268,274,0.962663,432,0.931454,474,0.969692,568,0.943406,421,0.947229,134,0.948536,618,0.945898,820,0.960007,710,0.948079,359,0.919464,480,0.941222,788
4,0.955385,387,0.964488,603,0.951574,371,0.957345,910,0.949066,812,0.959044,650,0.948761,342,0.948907,352,0.961868,787,0.927579,471,0.968738,644,0.943062,454,0.930759,135,0.946886,811,0.944011,901,0.958161,552,0.947159,371,0.917415,106,0.934217,266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,0.527776,908,0.594958,758,0.517116,543,0.556594,162,0.489921,53,0.630038,399,0.525574,614,0.596854,155,0.564816,270,0.366309,857,0.607311,625,0.312012,751,0.386514,671,0.454539,102,0.495088,553,0.545953,550,0.532684,443,0.295748,247,0.390677,70
976,0.527257,443,0.579246,391,0.511864,504,0.537721,244,0.472795,611,0.622525,611,0.510501,317,0.593838,614,0.558384,661,0.365965,758,0.594426,508,0.295005,55,0.384270,833,0.425003,671,0.492748,440,0.542373,478,0.522234,235,0.282196,883,0.380051,422
977,0.517417,883,0.576373,589,0.500099,903,0.507753,751,0.461575,797,0.617820,726,0.501840,859,0.586760,671,0.553330,614,0.346880,726,0.589096,399,0.292945,758,0.345135,804,0.417246,624,0.481648,874,0.526464,508,0.505446,370,0.264717,735,0.345456,797
978,0.498280,826,0.559972,399,0.486585,449,0.500050,83,0.415278,589,0.604674,883,0.500199,540,0.575294,317,0.551109,317,0.334156,874,0.584644,669,0.277935,874,0.331220,829,0.411338,638,0.478334,162,0.518248,399,0.504885,859,0.253405,171,0.339077,240


## Let's Try our first method which is using the modified weighted composite score.

In [8]:
new_users = user_data[0:19]
old_users = user_data[20:]

user_data.columns

Index(['user_id', 'investment_goal_long_term_growth',
       'investment_goal_stable_income', 'investment_goal_balanced_approach',
       'investment_goal_high_risk_high_reward',
       'investment_goal_short_term_trading', 'risk_tolerance',
       'holding_period_response', 'holding_period_reaction_strength',
       'expected_return_score', 'product_preference_stocks',
       'product_preference_etfs', 'product_preference_government_bonds',
       'product_preference_corporate_bonds',
       'product_preference_municipal_bonds',
       'product_preference_savings_bonds', 'product_preference_derivatives',
       'product_preference_alternative_investments',
       'factor_weight_investment_goals', 'factor_weight_risk_tolerance',
       'factor_weight_holding_period', 'factor_weight_return_score',
       'factor_weight_product_preference'],
      dtype='object')

In [9]:
user_data.head()
user_data.columns

Index(['user_id', 'investment_goal_long_term_growth',
       'investment_goal_stable_income', 'investment_goal_balanced_approach',
       'investment_goal_high_risk_high_reward',
       'investment_goal_short_term_trading', 'risk_tolerance',
       'holding_period_response', 'holding_period_reaction_strength',
       'expected_return_score', 'product_preference_stocks',
       'product_preference_etfs', 'product_preference_government_bonds',
       'product_preference_corporate_bonds',
       'product_preference_municipal_bonds',
       'product_preference_savings_bonds', 'product_preference_derivatives',
       'product_preference_alternative_investments',
       'factor_weight_investment_goals', 'factor_weight_risk_tolerance',
       'factor_weight_holding_period', 'factor_weight_return_score',
       'factor_weight_product_preference'],
      dtype='object')

In [72]:
user_data[["factor_weight_investment_goals", "factor_weight_risk_tolerance","factor_weight_holding_period", "factor_weight_return_score", "factor_weight_product_preference", "factor_weight_allocation_strength"]]

,factor_weight_investment_goals,factor_weight_risk_tolerance,factor_weight_holding_period,factor_weight_return_score,factor_weight_product_preference,factor_weight_allocation_strength
0,21.42,6.29,32.24,15.29,12.53,12.23
1,5.34,22.63,29.73,28.68,5.09,8.53
2,20.11,15.78,31.19,3.16,18.29,11.47
3,17.05,15.27,18.77,13.19,5.16,30.56
4,19.10,13.64,18.19,22.68,14.52,11.87
...,...,...,...,...,...,...
995,13.37,11.33,35.23,8.64,24.92,6.51
996,18.28,12.42,22.60,15.63,24.04,7.03
997,28.27,12.32,25.71,2.79,22.67,8.24
998,23.43,18.89,21.40,19.84,13.00,3.44
